
## transformer 实现
    1. word embedding 
    2. positional encodeing 
    3. encoder self-attention mask
    4. intra attention mask
    5. decoder self-attention mask
    6. multi-head self-attention 



#### [code scr: 【19、Transformer模型Encoder原理精讲及其PyTorch逐行实现】](https://space.bilibili.com/373596439)


In [3]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F



## word embedding

In [4]:
## 1. word embedding  序列建模
## source sentence 和 target sentence
## 构建序列, 序列的字符以索引的形式展示
batch_size = 2
max_num_src_words = 8  # 词表长度
max_num_tgt_words = 8  # 词表长度
max_src_seq_len = 5  ## 定义序列最大长度
max_tgt_seq_len = 5
model_dim = 8  # ,embedding 后的向量长度
max_position_len = 5  ## 序列编码的长度PE
# src_len = torch.randint(2, 5, size=(batch_size,))
# tgt_len = torch.randint(2, 5, size=(batch_size,))
src_len = torch.Tensor([2, 4]).to(torch.float32)  ## 原序列的两个句子的长度,2,4
tgt_len = torch.Tensor([4, 3]).to(torch.float32)  ## 目标序列的两个句子的长度,4,3
src_seq = [torch.randint(1, max_num_src_words, (L.to(torch.int),)) for L in src_len]  ## 单词索引构成的句子序列

tgt_seq = [torch.randint(1, max_num_tgt_words, (L.to(torch.int),)) for L in tgt_len]  ## 单词索引构成的句子序列 构建batch=> padding => embeddingF
print(f"未被填充的初始句子序列:\n{src_seq,tgt_seq}\n")
src_seq = [F.pad(L, (0, int(max(src_len) - len(L)))) for L in src_seq]
tgt_seq = [F.pad(L, (0, int(max(tgt_len) - len(L)))) for L in tgt_seq]
print(f"被填充后的句子序列:\n{src_seq,tgt_seq}\n")
## 将 tensor列表转为2维tensor
src_seq = torch.cat([torch.unsqueeze(L, dim=0) for L in src_seq], dim=0)
tgt_seq = torch.cat([torch.unsqueeze(L, dim=0) for L in tgt_seq], dim=0)
print(f"合并后的二维张量:\n{src_seq}\n{tgt_seq}\n")

## 构造embedding
src_embedding_table = nn.Embedding(num_embeddings=(max_num_src_words + 1), embedding_dim=model_dim)  ## 词表长度还要再加上"0"这个填充字符
tgt_embedding_table = nn.Embedding(num_embeddings=(max_num_tgt_words + 1), embedding_dim=model_dim)  ## 词表长度还要再加上"0"这个填充字符

print(f"tgt_embedding_table的内容,一行为一个字符的数字(字符)对应的embedding向量:\n{src_embedding_table.weight[0]}\n")
## 将src序列embedding化
src_embedding = src_embedding_table(src_seq)  # torch.Size([2, 5, 8])
## 将tgt序列embedding化
tgt_embedding = src_embedding_table(tgt_seq)  # torch.Size([2, 5, 8])


未被填充的初始句子序列:
([tensor([1, 6]), tensor([3, 5, 1, 1])], [tensor([4, 5, 5, 1]), tensor([5, 5, 1])])

被填充后的句子序列:
([tensor([1, 6, 0, 0]), tensor([3, 5, 1, 1])], [tensor([4, 5, 5, 1]), tensor([5, 5, 1, 0])])

合并后的二维张量:
tensor([[1, 6, 0, 0],
        [3, 5, 1, 1]])
tensor([[4, 5, 5, 1],
        [5, 5, 1, 0]])

tgt_embedding_table的内容,一行为一个字符的数字(字符)对应的embedding向量:
tensor([ 0.4662,  1.0600, -0.4784,  0.2710,  0.7075,  0.8951, -0.0653,  2.0116],
       grad_fn=<SelectBackward0>)



## pos embedding 
$$
\begin{aligned}
P E_{(p o s, 2 i)} & =\sin (\frac{p o s}{10000^{2 i / d_{\text {model }}}}) \\

P E_{(p o s, 2 i+1)} & =\cos (\frac{p o s } {10000^{2 i / d_{\text {mode }}}})
\end{aligned}
$$

In [14]:
# 2. 构造positional embedding  ## 矩阵的元素乘法+广播

post_mat = torch.arange(max_position_len).reshape(-1, 1)  ## position matrix
i_mat = torch.pow(10000, torch.arange(0, 8, 2).reshape(1, -1) / model_dim)  # 构造10000^(2i/dmodel)
pe_embedding_table = torch.zeros(max_position_len, model_dim)
pe_embedding_table[:, 0::2] = torch.sin(post_mat / i_mat)  # 处理偶数列
pe_embedding_table[:, 1::2] = torch.cos(post_mat / i_mat)  # 处理偶数列

## 我们想通过位置信息直接获取 ==>构建一个pos embedding 对象
pe_embedding = nn.Embedding(max_position_len, model_dim)
pe_embedding.weight = nn.Parameter(pe_embedding_table, requires_grad=False)  ## 手动填充参数,并不求梯度

## 生成src和tgt矩阵的位置索引矩阵
src_pos = torch.cat([torch.unsqueeze(torch.Tensor(torch.arange(max(src_len))).to(torch.int32), dim=0) for _ in src_len], dim=0)
tgt_pos = torch.cat([torch.unsqueeze(torch.Tensor(torch.arange(max(tgt_len))).to(torch.int32), dim=0) for _ in tgt_len], dim=0)
# print(post_mat, i_mat, pe_embedding_table,pe_embedding.weight)

print(tgt_pos)
print("----")
print(pe_embedding(src_pos))


tensor([[0, 1, 2, 3],
        [0, 1, 2, 3]], dtype=torch.int32)
----
tensor([[[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,
           1.0000e+00,  0.0000e+00,  1.0000e+00],
         [ 8.4147e-01,  5.4030e-01,  9.9833e-02,  9.9500e-01,  9.9998e-03,
           9.9995e-01,  1.0000e-03,  1.0000e+00],
         [ 9.0930e-01, -4.1615e-01,  1.9867e-01,  9.8007e-01,  1.9999e-02,
           9.9980e-01,  2.0000e-03,  1.0000e+00],
         [ 1.4112e-01, -9.8999e-01,  2.9552e-01,  9.5534e-01,  2.9995e-02,
           9.9955e-01,  3.0000e-03,  1.0000e+00]],

        [[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,
           1.0000e+00,  0.0000e+00,  1.0000e+00],
         [ 8.4147e-01,  5.4030e-01,  9.9833e-02,  9.9500e-01,  9.9998e-03,
           9.9995e-01,  1.0000e-03,  1.0000e+00],
         [ 9.0930e-01, -4.1615e-01,  1.9867e-01,  9.8007e-01,  1.9999e-02,
           9.9980e-01,  2.0000e-03,  1.0000e+00],
         [ 1.4112e-01, -9.8999e-01,  2.9552e-01,  9.55

## self attention mask <!-- markdownlint-disable MD000 -->

In [15]:
## 3. 构造encoder的self attention mask
## 这里使用的是先让Q.dot(K)再乘总masked matrix   //先对q,k矩阵乘法，以后再padding mask
## 也可以使用(Q*Q_mask).dot(K*K_mask)
valid_encoder_pos = torch.cat([torch.unsqueeze(
                                        F.pad(torch.ones(size=(int(L),)), (0, int(max(src_len) - L))), 
                                        dim=0)
                                        for L in src_len])  ## shape= torch.Size([2, 4])
valid_encoder_pos = torch.unsqueeze(valid_encoder_pos, dim=2)  ## 继续升维度 为bmm做准备  torch.Size([2, 4, 1])
## 两向量的矩阵相乘,可以获得两个向量的关联性
valid_encoder_pos_mat = torch.bmm(valid_encoder_pos, valid_encoder_pos.transpose(dim0=-1, dim1=-2))  ## shape=torch.Size([2, 4, 4])
# tensor([[[1., 1., 0., 0.],  ## 第一个词可以和第二词算相关性, 三四不行
#          [1., 1., 0., 0.],  ## 第2个词可以和第1词算相关性, 三四不行
#          [0., 0., 0., 0.],  ## 为padding的值,无法算相关性
#          [0., 0., 0., 0.]], ## 为padding的值,无法算相关性

#         [[1., 1., 1., 1.],  ## 均可计算
#          [1., 1., 1., 1.],
#          [1., 1., 1., 1.],
#          [1., 1., 1., 1.]]])
invalid_encoder_pos_mat = 1 - valid_encoder_pos_mat  # 反置
mask_encoder_self_attention = invalid_encoder_pos_mat.to(torch.bool)  ## 转为booL
## test
score = torch.randn(batch_size, int(max(src_len)), int(max(src_len)))
masked_score = score.masked_fill(mask_encoder_self_attention, -1e9)
prob = F.softmax(masked_score, dim=-1)

print(f"src_len\n{src_len}\n")
print(f"score\n{score}\n")
valid_encoder_pos.shape


src_len
tensor([2., 4.])

score
tensor([[[-0.4083,  0.0188,  1.4871, -0.5122],
         [-0.1784, -0.0745,  0.3354, -0.6136],
         [ 1.0952,  0.5340,  1.0339,  1.7858],
         [-1.5370, -0.1001,  0.0446,  1.7523]],

        [[-1.9907, -2.6149, -0.5830, -1.8669],
         [ 0.0241,  0.8287, -0.2836, -0.6909],
         [ 0.8815, -0.3933, -0.5205, -0.9167],
         [ 0.0904, -0.6645, -1.7895,  1.3866]]])



torch.Size([2, 4, 1])

In [8]:
print(f"masked_score\n{masked_score}\n")
print(f"prob\n{prob}\n")

masked_score
tensor([[[-1.2010e+00, -8.4091e-01, -1.0000e+09, -1.0000e+09],
         [ 8.0527e-02, -6.1755e-01, -1.0000e+09, -1.0000e+09],
         [-1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09],
         [-1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09]],

        [[-6.2462e-01,  4.5227e-01,  1.5756e+00,  5.8625e-01],
         [-8.6443e-01, -9.6584e-01, -2.2634e+00,  1.6207e-01],
         [ 6.8414e-01,  4.0627e-01, -3.8104e-01,  2.0532e-01],
         [-8.3061e-01, -5.9975e-01, -1.3869e-01, -1.2203e+00]]])

prob
tensor([[[0.4109, 0.5891, 0.0000, 0.0000],
         [0.6678, 0.3322, 0.0000, 0.0000],
         [0.2500, 0.2500, 0.2500, 0.2500],
         [0.2500, 0.2500, 0.2500, 0.2500]],

        [[0.0613, 0.1799, 0.5532, 0.2057],
         [0.2024, 0.1828, 0.0500, 0.5648],
         [0.3674, 0.2783, 0.1266, 0.2276],
         [0.2027, 0.2553, 0.4048, 0.1373]]])



## cross attention

In [16]:
##  4.intra self-attention mask  ## cross attention partment
## Q@K.T shape= [batch_size,tgt_seq_len , src_seq_len]
## 构造 valid encoder部分的有效位矩阵表示 (K的有效位表示)
valid_encoder_pos = torch.cat(  ## shape= torch.Size([2, 4])
    [
        torch.unsqueeze(
            F.pad(torch.ones(size=(int(L),)), (0, int(max(src_len) - L))), dim=0
        )
        for L in src_len
    ]
)
valid_encoder_pos = torch.unsqueeze(  ## shape= torch.Size([2, 4, 1])
    valid_encoder_pos, dim=2
)
## 构造 valid decoder部分的有效位矩阵表示 (Q的有效位表示)
valid_decoder_pos = torch.cat(  ## shape= torch.Size([2, 4])
    [
        torch.unsqueeze(
            F.pad(torch.ones(size=(int(L),)), (0, int(max(src_len) - L))), dim=0
        )
        for L in tgt_len
    ]
)
valid_decoder_pos = torch.unsqueeze(  ## shape= torch.Size([2, 4, 1])
    valid_decoder_pos, dim=2
)
## 通过bmm来获得两个矩阵相乘后的有效位表示矩阵  ## 原序列与目标序列是否要进行相关性运算
valid_cross_pos_mtx = torch.bmm(
    valid_decoder_pos, valid_encoder_pos.transpose(dim0=1, dim1=2)  ## 目标序列  ##  原序列
)  ## shape=torch.Size([2, 4, 4])
## (2,4,1)@ (2,1,4)  ## 表示只要两个有一个0这个位置就不用计算了==>置零
"""
tensor([[[1., 1., 0., 0.],  ## 第一个目标句子中的第一个单词对源句子中单词的有效性
          [1., 1., 0., 0.],
          [1., 1., 0., 0.],
          [1., 1., 0., 0.]],
 
         [[1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [0., 0., 0., 0.]]])
"""
invalid_cross_pos_mtx= 1-valid_cross_pos_mtx ## 反置
mask_cross_attention = invalid_cross_pos_mtx.to(torch.bool)  ## 转为booL
## 两向量的矩阵相乘,可以获得两个向量的关联性
# valid_encoder_pos, valid_decoder_pos,
mask_cross_attention


tensor([[[False, False,  True,  True],
         [False, False,  True,  True],
         [False, False,  True,  True],
         [False, False,  True,  True]],

        [[False, False, False, False],
         [False, False, False, False],
         [False, False, False, False],
         [ True,  True,  True,  True]]])

In [17]:
## test
score = torch.randn(batch_size, int(max(src_len)), int(max(tgt_len)))
masked_cross_score = score.masked_fill(mask_cross_attention, -1e9)
prob = F.softmax(masked_cross_score, dim=-1)

# print(f"src_len,tgt_len\n{src_len,tgt_len}\n")
# print(f"score\n{score}\n")
# valid_encoder_pos.shape
print(f"masked_cross_score\n{masked_cross_score}\n")
print(f"prob\n{prob}\n")

masked_cross_score
tensor([[[ 1.5032e+00,  6.4801e-01, -1.0000e+09, -1.0000e+09],
         [-8.8482e-01, -3.6397e-01, -1.0000e+09, -1.0000e+09],
         [-2.1182e-01,  5.6050e-01, -1.0000e+09, -1.0000e+09],
         [-5.0174e-01, -3.0729e-01, -1.0000e+09, -1.0000e+09]],

        [[-9.1142e-01, -6.1936e-01,  5.7152e-02,  1.4386e+00],
         [ 8.9409e-01, -5.1706e-01, -1.0612e+00,  1.2124e+00],
         [-1.0513e+00,  4.0985e-01, -1.0391e-01,  6.3027e-01],
         [-1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09]]])

prob
tensor([[[0.7017, 0.2983, 0.0000, 0.0000],
         [0.3727, 0.6273, 0.0000, 0.0000],
         [0.3160, 0.6840, 0.0000, 0.0000],
         [0.4515, 0.5485, 0.0000, 0.0000]],

        [[0.0647, 0.0866, 0.1704, 0.6783],
         [0.3623, 0.0884, 0.0513, 0.4981],
         [0.0754, 0.3250, 0.1944, 0.4052],
         [0.2500, 0.2500, 0.2500, 0.2500]]])



## decoder self attention mask

In [39]:
# 5. decoder self attention mask
valid_decoder_tri_mtx = [  ## 填充到batch内形状相同
    F.pad(
        torch.tril(torch.ones(int(L), int(L))),
        (0, int(max(tgt_len) - int(L)), 0, int(max(tgt_len) - int(L))),
    )
    for L in tgt_len
]
valid_decoder_tri_mtx = torch.cat([torch.unsqueeze(T, dim=0) for T in valid_decoder_tri_mtx], dim=0)  ## batch 拼接
valid_decoder_tri_mtx  # shape= torch.Size([2, 4, 4])
"""
tensor([[[1., 0., 0., 0.],  ## 有效位用1填充
         [1., 1., 0., 0.],
         [1., 1., 1., 0.],
         [1., 1., 1., 1.]],

        [[1., 0., 0., 0.],
         [1., 1., 0., 0.],
         [1., 1., 1., 0.],
         [0., 0., 0., 0.]]])
"""
invalid_decoder_tri_mtx = 1 - valid_decoder_tri_mtx
invalid_decoder_tri_mtx = invalid_decoder_tri_mtx.to(torch.bool)
invalid_decoder_tri_mtx


tensor([[[False,  True,  True,  True],
         [False, False,  True,  True],
         [False, False, False,  True],
         [False, False, False, False]],

        [[False,  True,  True,  True],
         [False, False,  True,  True],
         [False, False, False,  True],
         [ True,  True,  True,  True]]])

In [42]:
## test
score = torch.randn(batch_size, int(max(tgt_len)), int(max(tgt_len)))
masked_decoder_score = score.masked_fill(invalid_decoder_tri_mtx, -1e9)
prob = F.softmax(masked_decoder_score, dim=-1)

# print(f"src_len,tgt_len\n{src_len,tgt_len}\n")
# print(f"score\n{score}\n")
# valid_encoder_pos.shape
print(f"masked_decoder_score\n{masked_decoder_score}\n")
print(f"prob\n{prob}\n")

masked_decoder_score
tensor([[[-1.0629e+00, -1.0000e+09, -1.0000e+09, -1.0000e+09],
         [ 2.5091e-01, -1.5055e-01, -1.0000e+09, -1.0000e+09],
         [ 1.0840e+00, -6.6817e-01,  1.0421e+00, -1.0000e+09],
         [ 4.8028e-01, -4.0068e-01, -1.7346e-01, -7.4785e-01]],

        [[ 6.0432e-01, -1.0000e+09, -1.0000e+09, -1.0000e+09],
         [ 4.6865e-01,  1.1041e+00, -1.0000e+09, -1.0000e+09],
         [ 3.4363e-01,  2.7295e-01,  2.8848e-01, -1.0000e+09],
         [-1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09]]])

prob
tensor([[[1.0000, 0.0000, 0.0000, 0.0000],
         [0.5990, 0.4010, 0.0000, 0.0000],
         [0.4690, 0.0813, 0.4497, 0.0000],
         [0.4490, 0.1860, 0.2335, 0.1315]],

        [[1.0000, 0.0000, 0.0000, 0.0000],
         [0.3463, 0.6537, 0.0000, 0.0000],
         [0.3474, 0.3237, 0.3288, 0.0000],
         [0.2500, 0.2500, 0.2500, 0.2500]]])



In [ ]:
## 构建self attention
def scaled_dot_product_attention(Q, K, V, attn_mask):
    # shape of Q,K,V || Q:( batch_size*num_head,tgt_len,model_dim/num_head),,K:( batch_size*num_head,seq_len,model_dim/num_head)
    score = torch.bmm(Q, V.transpose(-2, -1)) / torch.sqrt(
        model_dim
    )  ## Q@K.T /model_dim
    masked_score = score.masked_fill(attn_mask, -1e9)  ## 填充被mask的位置
    prob = F.softmax(masked_score, dim=-1)  ## softmax处理
    context = torch.bmm(prob, V)  ## prob@ V
    return context


In [72]:
## loss
logits = torch.randn(2, 3, 4)  #   batch_size, seq_len, vocab_size
logits = logits.transpose(1, 2)
label = torch.randint(0, 4, (2, 3))
loss = F.cross_entropy(
    input=logits,
    target=label,
    reduction="none",
)
loss
tgt_len = torch.Tensor([2, 3]).to(torch.int32)  ##
# 生成对应的mask矩阵
loss_mask= torch.cat(
    [
        torch.unsqueeze(F.pad(torch.ones(L), (0, int(max(tgt_len)) - L)), dim=0)
        for L in tgt_len
    ],
    dim=0,
)
masked_loss_mtx= loss*loss_mask
print(masked_loss_mtx)

## 利用APi进行实现masked_loss
# ingnore_index：预测出的某个类别被忽略掉，不对反向传播和梯度更新产生贡献  默认值-100
label[0,2]=-100
print(label)
loss = F.cross_entropy(
    input=logits,
    target=label,
    reduction="none",
)
loss

tensor([[1.7158, 1.8576, 0.0000],
        [1.7036, 2.9287, 4.2849]])
tensor([[   3,    1, -100],
        [   3,    1,    2]])


tensor([[1.7158, 1.8576, 0.0000],
        [1.7036, 2.9287, 4.2849]])